<a href="https://colab.research.google.com/github/PARZIWALL/EDA-and-Clustering-/blob/main/Gagan_Sharma_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: Customers.csv:
# https://drive.google.com/file/d/1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE/view?usp=sharing
# Products.csv :
# https://drive.google.com/file/d/1IKuDizVapw-hyktwfpoAoaGtHtTNHfd0/view?usp=sharing
# Transactions.csv :
# https://drive.google.com/file/d/1saEqdbBB-vuk2hxoAf4TzDEsykdKlzbF/view?usp=sharing
# downLOad these datasets

!wget https://drive.google.com/uc?id=1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE -O Customers.csv
!wget https://drive.google.com/uc?id=1IKuDizVapw-hyktwfpoAoaGtHtTNHfd0 -O Products.csv
!wget https://drive.google.com/uc?id=1saEqdbBB-vuk2hxoAf4TzDEsykdKlzbF -O Transactions.csv


--2025-01-23 16:24:56--  https://drive.google.com/uc?id=1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE
Resolving drive.google.com (drive.google.com)... 74.125.69.113, 74.125.69.102, 74.125.69.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.69.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE [following]
--2025-01-23 16:24:56--  https://drive.usercontent.google.com/download?id=1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 64.233.179.132, 2607:f8b0:4001:c68::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|64.233.179.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8542 (8.3K) [application/octet-stream]
Saving to: ‘Customers.csv’

Customers.csv       100%[===================>]   8.34K  --.-KB/s    in 0s      

2025-01-23 16:24:59 (65.1 MB/s) - ‘Cust

In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Load the datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

# Merge the datasets: Transactions -> Customers -> Products
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

# Feature Engineering: Calculate Total Spend, Transaction Count (Frequency), and Account Age
total_spend = merged_data.groupby('CustomerID')['TotalValue'].sum()
transaction_count = merged_data.groupby('CustomerID')['TransactionID'].count()
account_age = pd.to_datetime(customers['SignupDate']).apply(lambda x: (pd.to_datetime('today') - x).days)

# Merge features into a single dataframe
customer_features = pd.DataFrame({
    'TotalSpend': total_spend,
    'TransactionCount': transaction_count,
    'AccountAge': account_age
}).fillna(0)  # Fill missing values (if any) with 0

# Fit the KNN model to find the most similar customers
knn = NearestNeighbors(n_neighbors=4, metric='cosine')
knn.fit(customer_features)

# Get the top 3 lookalike customers for each customer (excluding the customer itself)
distances, indices = knn.kneighbors(customer_features)

# Generate recommendations for the first 20 customers
recommendations = []
for i, customer_id in enumerate(customer_features.index[:20]):
    similar_customers = indices[i][1:]  # Exclude the customer itself
    similarity_scores = 1 - distances[i][1:]  # Cosine similarity score (higher is better)

    # Create a row with the customer ID and their top 3 lookalikes with similarity scores
    rec_row = [customer_id]
    for j in range(3):
        rec_row.extend([customer_features.index[similar_customers[j]], similarity_scores[j]])

    recommendations.append(rec_row)

# Create a DataFrame for recommendations
lookalike_df = pd.DataFrame(recommendations, columns=['CustomerID', 'Lookalike1_ID', 'Similarity_Score1',
                                                      'Lookalike2_ID', 'Similarity_Score2',
                                                      'Lookalike3_ID', 'Similarity_Score3'])

# Save the result to Lookalike.csv
lookalike_df.to_csv('Lookalike.csv', index=False)

print("Lookalike.csv has been generated.")


Lookalike.csv has been generated.


In [3]:
print(pd.read_csv("Lookalike.csv"))

   CustomerID Lookalike1_ID  Similarity_Score1 Lookalike2_ID  \
0       C0001         C0004                1.0         C0195   
1       C0002         C0198                1.0         C0076   
2       C0003         C0067                1.0         C0005   
3       C0004         C0001                1.0         C0137   
4       C0005         C0021                1.0         C0178   
5       C0006         C0079                1.0         C0141   
6       C0007         C0023                1.0         C0045   
7       C0008         C0088                1.0         C0010   
8       C0009         C0094                1.0         C0049   
9       C0010         C0008                1.0         C0088   
10      C0011         C0165                1.0         C0012   
11      C0012         C0011                1.0         C0165   
12      C0013         C0023                1.0         C0007   
13      C0014         C0147                1.0         C0009   
14      C0015         C0174             